<a href="https://colab.research.google.com/github/gui-ms/cmp263ML/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando Libs:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Funções:

In [ ]:
# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

def load_data(csv_path):
    """Carrega o dataset Parquet"""
    print("📊 Carregando dados...")
    df = pd.read_csv(csv_path)
    print(f"✅ Dataset carregado: {df.shape[0]} linhas × {df.shape[1]} colunas")
    return df

def basic_info(df):
    """Informações básicas do dataset"""
    print("\n" + "="*50)
    print("📋 INFORMAÇÕES BÁSICAS")
    print("="*50)

    print(f"• Dimensões: {df.shape[0]} linhas × {df.shape[1]} colunas")
    print(f"• Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    print("\n📊 Tipos de dados:")
    print(df.dtypes.value_counts())

    print("\n🔍 Primeiras 5 linhas:")
    print(df.head())

    print("\n⏩ Últimas 5 linhas:")
    print(df.tail())

def missing_analysis(df):
    """Análise de valores missing"""
    print("\n" + "="*50)
    print("🔍 ANÁLISE DE VALORES MISSING")
    print("="*50)

    missing = df.isnull().sum()
    missing_percent = (missing / len(df)) * 100

    missing_df = pd.DataFrame({
        'Coluna': missing.index,
        'Valores Missing': missing.values,
        'Percentual (%)': missing_percent.values
    }).sort_values('Valores Missing', ascending=False)

    print(missing_df[missing_df['Valores Missing'] > 0])

    if missing.sum() == 0:
        print("✅ Nenhum valor missing encontrado!")
    else:
        # Visualização
        plt.figure(figsize=(10, 6))
        missing_df[missing_df['Valores Missing'] > 0].plot(
            x='Coluna', y='Percentual (%)', kind='bar',
            title='Percentual de Valores Missing por Coluna'
        )
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

def infinites_analysis(df):
    """Análise de valores infinitos"""
    print("\n" + "="*50)
    print("🔍 ANÁLISE DE VALORES INFINITOS")
    print("="*50)

    # Selecionar apenas colunas numéricas
    num_cols = df.select_dtypes(include=[np.number]).columns

    # Criar uma máscara booleana para detectar valores infinitos
    mask_inf = np.isinf(df[num_cols])

    # Filtrar linhas que possuem pelo menos um valor infinito
    df_inf = df[mask_inf.any(axis=1)]

    # ---- Parte 1: visualizar dados infinitos ----
    print(f"Número de linhas com infinito: {df_inf.shape[0]}")
    print(f"Colunas com infinito: {list(mask_inf.any()[mask_inf.any()].index)}")

    #Taxa de valores inf
    df_inf[[ 'Flow Bytes/s', 'Flow Packets/s', 'Label']][df_inf.Label==1].count()/df_inf[[ 'Flow Packets/s', 'Label']].count()

    # ---- Parte 2: tratar os infinitos substituindo pelo maior valor finito da coluna ----
    for col in num_cols:
        if np.isinf(df[col]).any():
            max_val = df.loc[~np.isinf(df[col]), col].max()
            df[col] = df[col].replace([np.inf, -np.inf], max_val)
            print(f" Coluna '{col}' corrigida: ∞ substituído por {max_val}")

    # Agora df está limpo
    print("\n Dataset final corrigido!")

def nan_analysis(df):
    """Análise de valores nulos"""
    print("\n" + "="*50)
    print("🔍 ANÁLISE DE VALORES NULOS")
    print("="*50)

    # Taxa de valores ausentes por label com valor 1
    print('Taxa de valores ausentes:',df['Label'][df['Flow Bytes/s'].isna()][df['Label']==1].count()/df['Label'][df['Flow Bytes/s'].isna()].count())
    #tratamento
    print('substituição:',df['Flow Bytes/s'].max())
    df['Flow Bytes/s'] = df['Flow Bytes/s'].replace([np.nan], df['Flow Bytes/s'].max())
    print('Valores ausentes finais:',df['Label'][df['Flow Bytes/s'].isna()])


def num_cols_to_categorical(df):
    """Transforma colunas numéricas em categóricas"""
    for coluna in ['Src IP','Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp']:
      df[coluna] = df[coluna].astype('category')

# Função main:

In [ ]:
def main():
    """Função principal"""
    # Caminho para seu arquivo csv
    csv_path = "./dataset_completo.csv"  # Altere para o caminho correto

    try:
        # Carregar dados
        df = load_data(csv_path)


    except FileNotFoundError:
        print(f"❌ Arquivo {csv_path} não encontrado!")
        print("Verifique o caminho do arquivo.")
    except Exception as e:
        print(f"❌ Erro durante a análise: {e}")

if __name__ == "__main__":
    main()

📊 Carregando dados...
❌ Arquivo ./dataset_completo.csv não encontrado!
Verifique o caminho do arquivo.


# Montar drive (opcional):

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful